In [1]:
import numpy as np
import lsst.sims.featureScheduler as fs
from lsst.sims.featureScheduler.observatory import Speed_observatory
import healpy as hp
%matplotlib inline

In [2]:
survey_length = 10  # days
# Define what we want the final visit ratio map to look like
survey_filters = ['u','g']
surveys = []

for f in survey_filters:
    bfs = []
    bfs.append(fs.Slewtime_basis_function_cost(filtername=f))
    bfs.append(fs.Visit_repeat_basis_function_cost(filtername=f,survey_filters=survey_filters))
    bfs.append(fs.Target_map_basis_function_cost(filtername=f, survey_filters=survey_filters))
    bfs.append(fs.Normalized_alt_basis_function_cost(filtername=f))
    bfs.append(fs.Hour_angle_basis_function_cost())
    bfs.append(fs.Depth_percentile_basis_function_cost())
    weights = np.array([3,1,1,1,1,1])
    surveys.append(fs.Simple_greedy_survey_fields_cost(bfs, weights, filtername=f, block_size= 10))

scheduler = fs.Core_scheduler_cost(surveys)
observatory = Speed_observatory()
observatory, scheduler, observations = fs.sim_runner(observatory, scheduler,
                                                     survey_length=survey_length,
                                                     filename='pairs_survey.db', delete_past=True)

/Users/elahesadatnaghib/Dropbox/Graduate/Research/Princeton/LSSTscheduler/LSST-Code/PythonCode/sims_featureScheduler/python/lsst/sims/featureScheduler/features.py:377: RuntimeWarning: invalid value encountered in arccos
  self.ha_limit = np.arccos(cosha)*12/np.pi
/Users/elahesadatnaghib/Dropbox/Graduate/Research/Princeton/LSSTscheduler/LSST-Code/PythonCode/sims_featureScheduler/python/lsst/sims/featureScheduler/features.py:421: RuntimeWarning: invalid value encountered in greater
  self.feature[np.where(self.feature > time_to_twilight)] = time_to_twilight
/Users/elahesadatnaghib/Dropbox/Graduate/Research/Princeton/LSSTscheduler/LSST-Code/PythonCode/sims_featureScheduler/python/lsst/sims/featureScheduler/features.py:424: RuntimeWarning: invalid value encountered in less
  good = np.where(current_ha < -self.ha_limit)
/Users/elahesadatnaghib/Dropbox/Graduate/Research/Princeton/LSSTscheduler/LSST-Code/PythonCode/sims_featureScheduler/python/lsst/sims/featureScheduler/features.py:432: Runti

progress = 33.0%

/Users/elahesadatnaghib/Dropbox/Graduate/Research/Princeton/LSSTscheduler/LSST-Code/PythonCode/sims_featureScheduler/python/lsst/sims/featureScheduler/basis_functions.py:520: RuntimeWarning: divide by zero encountered in divide
  self.result = utils.alt_allocation(self.alt,self.dec, self.lat, self.filtername) + 2*((1./(1-np.cos(self.alt))) -1)


progress = 83.0%

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:

for fn, survey in zip(['u', 'g'], scheduler.surveys):
    for basis_function in survey.basis_functions:
        reward = basis_function()
        if np.size(reward) > 1:
            hp.mollview(reward, min=-1, title='$%s$, %s' % (fn, type(basis_function).__name__.split('.')[-1]))